In [1]:
pip install requests
pip install selenium
pip install beautifulsoup4 openpyxl
pip install webdriver-manager
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [86]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
import re
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [87]:
import openpyxl

In [88]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [89]:
# Chrome 옵션 설정
options = Options()

In [90]:
# WebDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [91]:
# 파일 경로 설정
file_path = '제품 가격zz.xlsx'

# 엑셀 파일 열기
wb = openpyxl.load_workbook(file_path)
ws = wb.active

# 품목명 읽기
product_names = [cell.value for cell in ws['B'][1:]] 
# 한글만 추출
korean_product_names = [re.sub('[^가-힣]', '', name) for name in product_names if name is not None]

In [92]:
# 웹사이트 로그인 페이지 열기
driver.get("https://www.dalrat.com/admin")
time.sleep(2)  # 실제 페이지 로드 시간에 따라 조정

# 로그인 필드에 자격 증명 입력 (가정)
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "login")
username.send_keys("dalrat")
password.send_keys("dalrat456!!")
password.send_keys(Keys.RETURN)
time.sleep(10)  # 실제 페이지 로드 시간에 따라 조정

In [93]:
# 상품페이지로 이동
driver.get("https://www.dalrat.com/admin/catalog/product/index/key/bc2441971289517038d420d5cf06b27ed1badcdc4a735f065dedb084d7ce4321/")
time.sleep(2)

# Chrome 옵션 설정
options = Options()

In [94]:
for row, name in enumerate(korean_product_names, start=2):
    try:
        search_input = driver.find_element(By.ID, "fulltext")
        search_input.clear()
        search_input.send_keys(name)
        search_input.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"Error finding or using search input: {e}")
        continue

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.admin__control-thumbnail")))
    except TimeoutException:
        print(f"Timeout waiting for thumbnail image of product: {name}")
        continue

    try:
        img_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "img.admin__control-thumbnail")))
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "img.admin__control-thumbnail")))
        driver.execute_script("arguments[0].click();", img_element)
    except TimeoutException:
        print(f"Thumbnail not clickable or not visible for product: {name}")
        continue
    except Exception as e:
        print(f"Error clicking thumbnail: {e}")
        continue

    # 이미지 모달이 로드될 때까지 기다립니다.
    try:
        WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element(By.CSS_SELECTOR, "img.thumbnail-preview-image").get_attribute('src') != previous_img_src
        )
        modal_img_element = driver.find_element(By.CSS_SELECTOR, "img.thumbnail-preview-image")
        img_src = modal_img_element.get_attribute('src')
        previous_img_src = img_src  # 현재 src를 이전 src로 저장
    except TimeoutException:
        print(f"Timeout waiting for new modal image to be visible for product: {name}")
        continue
    except Exception as e:
        print(f"Error retrieving new modal image: {e}")
        continue

    try:
        response = requests.get(img_src)
        if response.status_code == 200:
            with open(f'image_{row}.png', 'wb') as file:
                file.write(response.content)
            ws['C' + str(row)].value = f'image_{row}.png'
        else:
            print(f"Failed to download image with response code {response.status_code} for product: {name}")
    except Exception as e:
        print(f"Error downloading image: {e}")
        continue

    time.sleep(3)

Timeout waiting for thumbnail image of product: 인도미미고랭컵
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not def

KeyboardInterrupt: 

In [97]:

for row, name in enumerate(korean_product_names, start=2):
    try:
        search_input = driver.find_element(By.ID, "fulltext")
        search_input.clear()
        search_input.send_keys(name)
        search_input.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"Error finding or using search input: {e}")
        continue

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.admin__control-thumbnail")))
    except TimeoutException:
        print(f"Timeout waiting for thumbnail image of product: {name}")
        continue

    try:
        img_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "img.admin__control-thumbnail")))
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "img.admin__control-thumbnail")))
        driver.execute_script("arguments[0].click();", img_element)
    except TimeoutException:
        print(f"Thumbnail not clickable or not visible for product: {name}")
        continue
    except Exception as e:
        print(f"Error clicking thumbnail: {e}")
        continue

    try:
        WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element(By.CSS_SELECTOR, "img.thumbnail-preview-image").get_attribute('src') != previous_img_src
        )
        modal_img_element = driver.find_element(By.CSS_SELECTOR, "img.thumbnail-preview-image")
        img_src = modal_img_element.get_attribute('src')
        previous_img_src = img_src  # 현재 src를 이전 src로 저장
    except TimeoutException:
        print(f"Timeout waiting for new modal image to be visible for product: {name}")
        continue
    except Exception as e:
        print(f"Error retrieving new modal image: {e}")
        continue

    # 팝업 창을 닫고 메인 윈도우로 돌아갑니다.
    driver.close()
    driver.switch_to.window(main_window_handle)

    time.sleep(3)  # 서버에 과부하를 주지 않기 위해 간단한 딜레이를 추가합니다.

Timeout waiting for thumbnail image of product: 인도미미고랭컵
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not defined
Error retrieving new modal image: name 'previous_img_src' is not def

In [96]:
wb.save('제품 가격zz.xlsx')


In [49]:
driver.quit()